### Почему «солёное» хэширование так хорошо?

"Солёное" хэширование — это ключевой инструмент при построении надёжной системы сплитования. Основные его преимущества:
- **Детерминированность:** один и тот же `user_id` всегда попадает в одну и ту же группу.
- **Случайность:** группы формируются случайным образом.
- **Равномерность:** группы примерно одинакового размера.

Это идеальное сочетание свойств для проведения A/B-тестов.

#### Пример реализации

#### 1. Подключение к ClickHouse и получение списка пользователей

In [ ]:
import pandahouse as ph
import swifter

In [ ]:
# Параметры подключения
connection = {
    'host': 'https://clickhouse.lab.karpov.courses ',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}

In [ ]:
# SQL-запрос: выбираем уникальных пользователей за неделю
query_users = """
SELECT DISTINCT user_id
FROM simulator.feed_actions 
WHERE toDate(time) >= '2021-11-15' AND toDate(time) <= '2021-11-21'
"""

# Загружаем пользователей
users_df = ph.read_clickhouse(query_users, connection=connection)
users_df.head()

#### 2. Разбиение на группы с помощью функции `ab_split`

In [ ]:
def ab_split(user_id, salt='my_salt', n_groups=5):
    import mmh3
    return mmh3.hash(f"{user_id}{salt}") % n_groups

In [ ]:
# Применяем функцию к user_id
users_df['hash_group'] = users_df['user_id'].swifter.apply(ab_split)

# Считаем количество пользователей в каждой группе
group_distribution = users_df.groupby('hash_group')['user_id'].nunique().reset_index()
group_distribution.columns = ['hash_group', 'user_count']

group_distribution

#### 3. Альтернатива: разбиение прямо в ClickHouse

In [ ]:
# SQL-запрос с хэшированием внутри ClickHouse
query_split = """
SELECT 
    xxHash64(toString(user_id) || 'my_salt') % 5 AS hash_group,
    uniqExact(user_id) AS user_count
FROM simulator.feed_actions 
WHERE toDate(time) >= '2021-11-15' AND toDate(time) <= '2021-11-21'
GROUP BY hash_group
ORDER BY hash_group
"""

# Выполняем запрос
split_result = ph.read_clickhouse(query_split, connection=connection)
split_result

### Вывод

"Солёное" хэширование:
- Гарантирует **равномерное распределение** пользователей по группам.
- Обеспечивает **воспроизводимость** результатов.
- Позволяет избежать пересечения экспериментов.

Использовать его можно как в Python (через библиотеки вроде `mmh3`), так и непосредственно в ClickHouse — в зависимости от ваших задач и предпочтений.

 *Совет: используй разные соли для разных экспериментов, чтобы гарантировать их независимость.*